In [ ]:
import geopandas
import urllib
import pandas
from datetime import date
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import contextily
import re

In [ ]:
import fiona;
fiona.supported_drivers


In [ ]:
# mapping_crs = "EPSG:3347" # NAD83 / Statistics Canada Lambert
mapping_crs = "EPSG:3857" # Pseudo-Mercator
output_crs = "EPSG:4326" # Lat/Lng https://epsg.io/4326

In [ ]:
def load_geodataset(remote_path, local_path, refresh_cache=False):
    df = None
    if not refresh_cache:
        try:
            df = geopandas.read_file(local_path)
        except Exception as e:
            print(e)
    
    if df is None:
        print(f"Loading {remote_path}")
        df = geopandas.read_file(remote_path)

        print(f"Saving to {local_path}")
        df.to_file(local_path)

    return df

In [ ]:
def get_vancouver_bike_map(refresh_cache=False):
    # Get City of Vancouver's bikeways dataset https://opendata.vancouver.ca/explore/dataset/bikeways/
    remote_path = "https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/bikeways/exports/geojson?lang=en&timezone=America%2FLos_Angeles"
    local_path = "data/bikeways.geojson"
    return load_geodataset(remote_path, local_path, refresh_cache=refresh_cache)


In [ ]:
pandas.set_option('display.max_columns', None)

In [ ]:
vancouver_bike_map = get_vancouver_bike_map()
vancouver_bike_map

In [ ]:
def calc_removed_year(row):
    if row["status"] == "Legacy":
        m = re.search(r"Removed in (\d{4})", row["notes"])
        if m:
            return int(m.group(1))
    return np.NaN


In [ ]:
vancouver_bike_map["year_removed"] = vancouver_bike_map.apply(calc_removed_year, axis=1)

In [ ]:
vancouver_bike_map = vancouver_bike_map[~vancouver_bike_map["year_of_construction"].isna()]
vancouver_bike_map = vancouver_bike_map.astype({"year_of_construction": "float", "upgrade_year": "float"})


In [ ]:
vancouver_bike_map.plot()

In [ ]:
vancouver_bike_map = vancouver_bike_map.to_crs(mapping_crs)

In [ ]:
full_map = vancouver_bike_map.plot(column="year_of_construction")

In [ ]:
full_map = vancouver_bike_map.plot(column="aaa_network")

In [ ]:
full_map = vancouver_bike_map.plot(column="aaa_segment")

In [ ]:
vancouver_bike_map[["bikeway_type", "aaa_segment", "segment_length"]].groupby(["bikeway_type", "aaa_segment"]).sum()

In [ ]:
contextily.providers

In [ ]:
blue_colors = matplotlib.colors.ListedColormap(["xkcd:bright blue", "xkcd:sky blue"])
blue_colors


In [ ]:
def filter_bike_map(year):
    # Figure out of the bike lane as AAA in this year
    def calc_current_status(row):
        if row["aaa_segment"] == "YES" and not (year < row["upgrade_year"]):
            return "AAA"
        return "standard"
    vancouver_bike_map["current_type"] = vancouver_bike_map.apply(calc_current_status, axis=1)

    filtered = vancouver_bike_map[(year >= vancouver_bike_map["year_of_construction"]) & ~(year >= vancouver_bike_map["year_removed"])]
    return filtered

In [ ]:
min_year = int(min(vancouver_bike_map["year_of_construction"]))
max_year = date.today().year

In [ ]:
bikeway_summary_by_year = []
for year in range(min_year, max_year+1):
    filtered = filter_bike_map(year)
    bikeway_summary_by_year.append({
        "year": year,
        "total": filtered["segment_length"].sum() / 1000,
        "AAA": filtered[filtered["current_type"] == "AAA"]["segment_length"].sum() / 1000,
        "standard": filtered[filtered["current_type"] == "standard"]["segment_length"].sum() / 1000,
    })
bikeway_summary_by_year = pandas.DataFrame(bikeway_summary_by_year).set_index("year")

In [ ]:
bikeway_summary_by_year

In [ ]:
bikeway_summary_by_year[["AAA", "standard"]].plot.area(stacked=True, cmap=blue_colors)

In [ ]:
def draw_map(bike_map, filename, subtitle=None, color_by=None, cmap=None, title="Vancouver Bike Map", summary=None):
    ax = bike_map.plot(figsize=(10, 10), column=color_by, cmap=cmap, legend=True)
    ax.set_xlim(full_map.get_xlim())
    ax.set_ylim(full_map.get_ylim())
    ax.set_axis_off()
    ax.text(1, 0, 'https://canadianveggie.com', transform=ax.transAxes,
        fontsize=10, color='gray', alpha=0.5,
        ha='right', va='bottom')
    # ax.get_xaxis().set_visible(False)
    # ax.get_yaxis().set_visible(False)
    plt.title(subtitle,fontsize=18, y=1)
    plt.suptitle(title,fontsize=30, y=0.95)
    contextily.add_basemap(ax, source=contextily.providers.CartoDB.Voyager)
    if summary is not None:
        axin1 = ax.inset_axes([0.072, 0.83, 0.25, 0.15])
        summary_chart = summary.plot.area(ax=axin1, legend=False, stacked=True, linewidth=0, cmap=blue_colors)
        summary_chart.set_ylim(0, 400)
        summary_chart.set_ylabel("kms")
    ax.get_figure().savefig(filename)
    plt.close('all')


In [ ]:
draw_map(vancouver_bike_map, f"output/all.png", color_by="bikeway_type")

In [ ]:
for year in range(min_year, max_year+1):
    print(year)
    filtered = filter_bike_map(year)
    summary = bikeway_summary_by_year[["AAA", "standard"]].copy()
    summary.loc[summary.index > year, 'AAA'] = np.nan
    summary.loc[summary.index > year, 'standard'] = np.nan
    draw_map(filtered, f"output/vancouver_bike_map_{year}.png", subtitle=str(year), color_by="current_type", cmap=blue_colors, summary=summary)
